### Bokeh for scatter geo map

Brian Dew, @bd_econ

----

Attempt to draw a US map and mark unemployment rate data by metro area.

In [1]:
#Import preliminaries
import requests
import numpy as np
import pandas as pd
import shapefile

# Bokeh settings
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource, HoverTool, Circle
from bokeh.embed import components
from bokeh.io import output_notebook, reset_output

output_notebook()

Loading BokehJS ...

In [2]:
def msa_calc(msa, data):
    '''Return labor market stats from BLS data when given BLS MSA code'''
    dfm = data[data['Code'] == msa]
    md = MSA[MSA['Code'] == msa]
    st = md['State'].iloc[-1]
    lt = dfm['Rate'].iloc[-1]
    ltd = dfm['date'].iloc[-1].strftime('%B %Y')
    m = dfm['Area'].iloc[-1]
    d12 = round(dfm.Rate.astype('float').diff(12).iloc[-1], 1)
    d3 = round(dfm.Rate.astype('float').rolling(3).mean().diff(3).iloc[-1], 1)
    d1 = round(dfm.Rate.astype('float').diff(1).iloc[-1], 1)
    c12 = ('limegreen' if d12 < -0.001 else 'red' if d12 > 0.001 else 'lightgray')
    c3 = ('limegreen' if d3 < -0.001 else 'red' if d3 > 0.001  else 'lightgray')
    c1 = ('limegreen' if d1 < -0.001 else 'red' if d1 > 0.001  else 'lightgray')
    s12 = (abs(d12) * 8.0 + 5 if abs(d12) > 0 else 5)
    s3 = (abs(d3) * 8.0 + 5 if abs(d3) > 0 else 5)
    s1 = (abs(d1) * 8.0 + 5 if abs(d1) > 0 else 5)
    lat = md['latitude'].iloc[-1]
    lon = md['longitude'].iloc[-1]
    if st == 'AK':
        lat = 0.35 * md['latitude'].iloc[-1] + 5
        lon = 0.35 * md['longitude'].iloc[-1] - 63
    if st == 'HI':
        lat = md['latitude'].iloc[-1] + 7
        lon = md['longitude'].iloc[-1] + 51
    sh12 = ('▼' if d12 < -0.001 else '▲' if d12 > 0.001 else '')
    sh3 = ('▼' if d3 < -0.001 else '▲' if d3 > 0.001 else '')
    sh1 = ('▼' if d1 < -0.001 else '▲' if d1 > 0.001 else '')
    return {'name': m, 'latest': lt, 'd12': d12, 'c12': c12, 's12': s12, 
            'd3': d3, 'c3': c3, 's3': s3, 'd1': d1, 'c1': c1, 's1': s1, 
            'lat': lat, 'lon': lon, 'date': ltd, 'sh12': sh12, 'sh3': sh3,
            'sh1': sh1}

In [7]:
# CSV file contains city names, their BLS codes (Metropolitan 
# MTAs/NECTAs only), and their latitude and longitude
MSA = pd.read_csv('C:/Working/Python/Unemp_Map/MSA.csv')
MSA['Code'] = [m[1]['fullcode'][3:-2] for m in MSA.iterrows()]
MSA['State'] = [m[1]['fullname'][-2:] for m in MSA.iterrows()]
MSA = MSA[MSA['State'] != 'PR']

url = 'https://www.bls.gov/web/metro/ssamatab1.txt'
#url = 'ssamatab1.txt'

df = pd.read_table(url, sep='\s\s+', skiprows=3, skipfooter=4, engine='python')
data = df[df['Year'] >= 2017]
data['date'] = [pd.to_datetime(f'{v.Year}-{v.Month}-01') for i, v in data.iterrows()]

result = pd.DataFrame([msa_calc(msa, data) for msa in list(MSA.Code.unique())])

C:\Users\BDew\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


In [8]:
# Get State Map Data
shpfile = 'C:/Working/Python/Unemp_Map/st99_d00.shp'
sf = shapefile.Reader(shpfile)
skip = ['Alaska', 'Hawaii', 'Puerto Rico', 'Tennessee', 'West Virginia', 'Iowa', 'Wyoming', 'Nevada', 'Oklahoma']

rows=[]
for i, j in zip(sf.shapeRecords(), sf.shapes()):
    if i.record[0] > 0.02:
        r_points = [j for j in j.points]
        rows.append(i.record + [r_points])

others = {f'{r[5]}{r[3]}': {'lats': [i[1] for i in r[-1]], 
                            'lons': [i[0] for i in r[-1]], 
                            'name': r[5]} for r in rows if r[5] not in skip}

alaska = {f'{r[5]}{r[3]}': {'lats': [0.35 * i[1] + 5 for i in r[-1]], 
                            'lons': [0.35 * i[0] - 63 for i in r[-1]], 
                            'name': r[5]} for r in rows if r[5] == 'Alaska' 
                              and np.max([i[0] for i in r[-1]]) < 0}
    
hawaii = {f'{r[5]}{r[3]}': {'lats': [i[1] + 7 for i in r[-1]], 
                            'lons': [i[0] + 51 for i in r[-1]], 
                            'name': r[5]} for r in rows if r[5] == 'Hawaii'}

states = {**others, **hawaii, **alaska}

In [9]:
# Three Month moving Average

# Identify data sources
source = ColumnDataSource(data=dict(
    x = [state["lons"] for state in states.values()],
    y = [state["lats"] for state in states.values()],
    name = [state['name'] for state in states.values()]))

# Matching up MSA table with bokeh format
source2 = ColumnDataSource(data=dict(
        lat = list(result['lat'].values),
        lon = list(result['lon'].values),
        s12 = list(result['s12'].values),
        s3 = list(result['s3'].values),
        s1 = list(result['s1'].values),
        name = list(result['name'].values),
        color_ann = list(result['c12'].values),
        color_q = list(result['c3'].values),
        color_m = list(result['c1'].values),
        lt_val = list(result['latest'].values),
        d_ann = list(result['d12'].values),
        d_q = list(result['d3'].values),
        d_m = list(result['d1'].values),
        date = list(result['date'].values),
        sh_ann = list(result['sh12'].values),
        sh_q = list(result['sh3'].values),
        sh_m = list(result['sh1'].values)))

# HTML text that shows when hovering over an MSA
tooltips = """
    <div style="line-height: 110%;">
        <span style="font-size: 12px;">@name:</span><br>
        <span style="font-size: 11px; font-style: italic;">@date:</span>
        <div style="text-align: right;">
        <span style="font-size: 10px; padding: 0 5px;">Unemployed, looking for work: </span>
        <span style="display: inline-block; font-size: 11px; font-weight: bold; min-width: 26px;">@lt_val{1.1}%</span><br>
        <span style="font-size: 10px; padding: 0 5px;">One-year change (pp): </span>
        <span style="display: inline-block; color: @color_ann; width: 12px;">@sh_ann</span>
        <span style="display: inline-block; font-size: 11px; font-weight: bold; padding: 0 8px 0 0; width: 26px;">@d_ann{1.1}</span><br>
        <span style="font-size: 10px; padding: 0 5px;">Quarterly change (pp): </span>
        <span style="display: inline-block; color: @color_q; width: 12px;">@sh_q</span>
        <span style="display: inline-block; font-size: 11px; font-weight: bold; padding: 0 8px 0 0; width: 26px;">@d_q{1.1}</span><br>
        <span style="font-size: 10px; padding: 0 5px;">Monthly change (pp): </span>
        <span style="display: inline-block; color: @color_m; width: 12px;">@sh_m</span>
        <span style="display: inline-block; font-size: 11px; font-weight: bold; padding: 0 8px 0 0; width: 26px;">@d_m{1.1}</span></div>
    </div>
"""

hover = HoverTool(tooltips=tooltips, show_arrow=False, names=['MSA'])
TOOLS = 'pan, wheel_zoom, reset, save'

reset_output()

# Quarterly Change (Change over prev month)
p3 = figure(tools=[TOOLS, hover], plot_width=765, plot_height=486, toolbar_location="below",  
    x_axis_location=None, y_axis_location=None, output_backend='webgl')
p3.min_border = 0
p3.sizing_mode = 'scale_width'
p3.grid.grid_line_color = None
p3.outline_line_color = None
p3.patches('x', 'y', source=source, fill_color=None, line_color='darkslategray', line_width=0.5)

p3.circle(x="lon", y="lat", size="s3", fill_color="color_q", source=source2,
                fill_alpha=0.5, line_color="color_q", name='MSA')

show(p3)

script, div = components(p3)
with open('C:/Working/bdecon.github.io/plots/unemp_map_3m.html', 'w+') as text_file:
    text_file.write(f'{script} {div}')

In [10]:
# One year change (default)

# Identify data sources
source = ColumnDataSource(data=dict(
    x = [state["lons"] for state in states.values()],
    y = [state["lats"] for state in states.values()],
    name = [state['name'] for state in states.values()]))

# Matching up MSA table with bokeh format
source2 = ColumnDataSource(data=dict(
        lat = list(result['lat'].values),
        lon = list(result['lon'].values),
        s12 = list(result['s12'].values),
        s3 = list(result['s3'].values),
        s1 = list(result['s1'].values),
        name = list(result['name'].values),
        color_ann = list(result['c12'].values),
        color_q = list(result['c3'].values),
        color_m = list(result['c1'].values),
        lt_val = list(result['latest'].values),
        d_ann = list(result['d12'].values),
        d_q = list(result['d3'].values),
        d_m = list(result['d1'].values),
        date = list(result['date'].values),
        sh_ann = list(result['sh12'].values),
        sh_q = list(result['sh3'].values),
        sh_m = list(result['sh1'].values)))

# HTML text that shows when hovering over an MSA
tooltips = """
    <div style="line-height: 110%;">
        <span style="font-size: 12px;">@name:</span><br>
        <span style="font-size: 11px; font-style: italic;">@date:</span>
        <div style="text-align: right;">
        <span style="font-size: 10px; padding: 0 5px;">Unemployed, looking for work: </span>
        <span style="display: inline-block; font-size: 11px; font-weight: bold; min-width: 26px;">@lt_val{1.1}%</span><br>
        <span style="font-size: 10px; padding: 0 5px;">One-year change (pp): </span>
        <span style="display: inline-block; color: @color_ann; width: 12px;">@sh_ann</span>
        <span style="display: inline-block; font-size: 11px; font-weight: bold; padding: 0 8px 0 0; width: 26px;">@d_ann{1.1}</span><br>
        <span style="font-size: 10px; padding: 0 5px;">Quarterly change (pp): </span>
        <span style="display: inline-block; color: @color_q; width: 12px;">@sh_q</span>
        <span style="display: inline-block; font-size: 11px; font-weight: bold; padding: 0 8px 0 0; width: 26px;">@d_q{1.1}</span><br>
        <span style="font-size: 10px; padding: 0 5px;">Monthly change (pp): </span>
        <span style="display: inline-block; color: @color_m; width: 12px;">@sh_m</span>
        <span style="display: inline-block; font-size: 11px; font-weight: bold; padding: 0 8px 0 0; width: 26px;">@d_m{1.1}</span></div>
    </div>
"""

hover = HoverTool(tooltips=tooltips, show_arrow=False, names=['MSA'])
TOOLS = 'pan, wheel_zoom, reset, save'

reset_output()

# Quarterly Change (Change over prev month)
p12 = figure(tools=[TOOLS, hover], plot_width=765, plot_height=486, toolbar_location="below",  
    x_axis_location=None, y_axis_location=None, output_backend='webgl')
p12.min_border = 0
p12.sizing_mode = 'scale_width'
p12.grid.grid_line_color = None
p12.outline_line_color = None
p12.patches('x', 'y', source=source, fill_color=None, line_color='darkslategray', line_width=0.5)

p12.circle(x="lon", y="lat", size="s12", fill_color="color_ann", source=source2,
                fill_alpha=0.5, line_color="color_ann", name='MSA')

show(p12)

script, div = components(p12)
with open('C:/Working/bdecon.github.io/plots/unemp_map_12m.html', 'w+') as text_file:
    text_file.write(f'{script} {div}')

In [11]:
# One month change 

# Identify data sources
source = ColumnDataSource(data=dict(
    x = [state["lons"] for state in states.values()],
    y = [state["lats"] for state in states.values()],
    name = [state['name'] for state in states.values()]))

# Matching up MSA table with bokeh format
source2 = ColumnDataSource(data=dict(
        lat = list(result['lat'].values),
        lon = list(result['lon'].values),
        s12 = list(result['s12'].values),
        s3 = list(result['s3'].values),
        s1 = list(result['s1'].values),
        name = list(result['name'].values),
        color_ann = list(result['c12'].values),
        color_q = list(result['c3'].values),
        color_m = list(result['c1'].values),
        lt_val = list(result['latest'].values),
        d_ann = list(result['d12'].values),
        d_q = list(result['d3'].values),
        d_m = list(result['d1'].values),
        date = list(result['date'].values),
        sh_ann = list(result['sh12'].values),
        sh_q = list(result['sh3'].values),
        sh_m = list(result['sh1'].values)))

# HTML text that shows when hovering over an MSA
tooltips = """
    <div style="line-height: 110%;">
        <span style="font-size: 12px;">@name:</span><br>
        <span style="font-size: 11px; font-style: italic;">@date:</span>
        <div style="text-align: right;">
        <span style="font-size: 10px; padding: 0 5px;">Unemployed, looking for work: </span>
        <span style="display: inline-block; font-size: 11px; font-weight: bold; min-width: 26px;">@lt_val{1.1}%</span><br>
        <span style="font-size: 10px; padding: 0 5px;">One-year change (pp): </span>
        <span style="display: inline-block; color: @color_ann; width: 12px;">@sh_ann</span>
        <span style="display: inline-block; font-size: 11px; font-weight: bold; padding: 0 8px 0 0; width: 26px;">@d_ann{1.1}</span><br>
        <span style="font-size: 10px; padding: 0 5px;">Quarterly change (pp): </span>
        <span style="display: inline-block; color: @color_q; width: 12px;">@sh_q</span>
        <span style="display: inline-block; font-size: 11px; font-weight: bold; padding: 0 8px 0 0; width: 26px;">@d_q{1.1}</span><br>
        <span style="font-size: 10px; padding: 0 5px;">Monthly change (pp): </span>
        <span style="display: inline-block; color: @color_m; width: 12px;">@sh_m</span>
        <span style="display: inline-block; font-size: 11px; font-weight: bold; padding: 0 8px 0 0; width: 26px;">@d_m{1.1}</span></div>
    </div>
"""

hover = HoverTool(tooltips=tooltips, show_arrow=False, names=['MSA'])
TOOLS = 'pan, wheel_zoom, reset, save'

reset_output()

# Quarterly Change (Change over prev month)
p1 = figure(tools=[TOOLS, hover], plot_width=765, plot_height=486, toolbar_location="below",  
    x_axis_location=None, y_axis_location=None, output_backend='webgl')
p1.min_border = 0
p1.sizing_mode = 'scale_width'
p1.grid.grid_line_color = None
p1.outline_line_color = None
p1.patches('x', 'y', source=source, fill_color=None, line_color='darkslategray', line_width=0.5)

p1.circle(x="lon", y="lat", size="s1", fill_color="color_m", source=source2,
                fill_alpha=0.5, line_color="color_m", name='MSA')

show(p1)

script, div = components(p1)
with open('C:/Working/bdecon.github.io/plots/unemp_map_1m.html', 'w+') as text_file:
    text_file.write(f'{script} {div}')

In [12]:
# generate html table with data
table = result[['name', 'latest', 'd12', 'd3', 'd1']]
table.columns = ['Metropolitan Statistical Area (MSA) Name', 'Unemp. rate', '12-mo. change', '3-mo. change', '1-mo. change']
table = table.sort_values('12-mo. change').drop_duplicates() 
# Sortable css class allows for js script to sort each column
# https://www.kryogenix.org/code/browser/sorttable/
table.to_html('C:/Working/bdecon.github.io/Dash/unemp_table.html', border=0, 
              index=False, classes='sortable')

In [ ]:
# Desktop version

# Identify data sources
source = ColumnDataSource(data=dict(
    x = [state["lons"] for state in states.values()],
    y = [state["lats"] for state in states.values()],
    name = [state['name'] for state in states.values()]))

# Matching up MSA table with bokeh format
source2 = ColumnDataSource(data=dict(
        lat = list(result['lat'].values),
        lon = list(result['lon'].values),
        s12 = list(result['s12'].values),
        s3 = list(result['s3'].values),
        s1 = list(result['s1'].values),
        name = list(result['name'].values),
        color_ann = list(result['c12'].values),
        color_q = list(result['c3'].values),
        color_m = list(result['c1'].values),
        lt_val = list(result['latest'].values),
        d_ann = list(result['d12'].values),
        d_q = list(result['d3'].values),
        d_m = list(result['d1'].values),
        date = list(result['date'].values),
        sh_ann = list(result['sh12'].values),
        sh_q = list(result['sh3'].values),
        sh_m = list(result['sh1'].values)))

# HTML text that shows when hovering over an MSA
tooltips = """
    <div style="line-height: 110%;">
        <span style="font-size: 12px;">@name:</span><br>
        <span style="font-size: 11px; font-style: italic;">@date:</span>
        <div style="text-align: right;">
        <span style="font-size: 10px; padding: 0 5px;">Unemployed, looking for work: </span>
        <span style="display: inline-block; font-size: 11px; font-weight: bold; min-width: 26px;">@lt_val{1.1}%</span><br>
        <span style="font-size: 10px; padding: 0 5px;">One-year change (pp): </span>
        <span style="display: inline-block; color: @color_ann; width: 12px;">@sh_ann</span>
        <span style="display: inline-block; font-size: 11px; font-weight: bold; padding: 0 8px 0 0; width: 26px;">@d_ann{1.1}</span><br>
        <span style="font-size: 10px; padding: 0 5px;">Quarterly change (pp): </span>
        <span style="display: inline-block; color: @color_q; width: 12px;">@sh_q</span>
        <span style="display: inline-block; font-size: 11px; font-weight: bold; padding: 0 8px 0 0; width: 26px;">@d_q{1.1}</span><br>
        <span style="font-size: 10px; padding: 0 5px;">Monthly change (pp): </span>
        <span style="display: inline-block; color: @color_m; width: 12px;">@sh_m</span>
        <span style="display: inline-block; font-size: 11px; font-weight: bold; padding: 0 8px 0 0; width: 26px;">@d_m{1.1}</span></div>
    </div>
"""

hover = HoverTool(tooltips=tooltips, show_arrow=False, names=['MSA'])
TOOLS = 'pan, wheel_zoom, reset, save'

# Annual change
p12 = figure(tools=[TOOLS, hover], plot_width=850, plot_height=540, toolbar_location="below",  
    x_axis_location=None, y_axis_location=None)
p12.sizing_mode = 'scale_width'
p12.grid.grid_line_color = None
p12.outline_line_color = None
p12.patches('x', 'y', source=source, fill_color='white',
          fill_alpha=1.0, line_color='darkslategray', line_width=0.5)

p12.circle(x="lon", y="lat", size="s12", fill_color="color_ann", source=source2,
                fill_alpha=0.5, line_color="color_ann", name='MSA')

# Quarterly Change (3m avg over prev 3m avg)
p3 = figure(tools=[TOOLS, hover], plot_width=850, plot_height=540, toolbar_location="below",  
    x_axis_location=None, y_axis_location=None)

p3.grid.grid_line_color = None
p3.outline_line_color = None
p3.patches('x', 'y', source=source, fill_color='white',
          fill_alpha=1.0, line_color='darkslategray', line_width=0.5)

p3.circle(x="lon", y="lat", size="s3", fill_color="color_q", source=source2,
                fill_alpha=0.5, line_color="color_q", name='MSA')

# Monthly change
p1 = figure(tools=[TOOLS, hover], plot_width=850, plot_height=540,  toolbar_location="below",  
    x_axis_location=None, y_axis_location=None)

p1.grid.grid_line_color = None
p1.outline_line_color = None 
p1.patches('x', 'y', source=source, fill_color='white',
          fill_alpha=1.0, line_color='darkslategray', line_width=0.5)

p1.circle(x="lon", y="lat", size="s1", fill_color="color_m", source=source2,
                fill_alpha=0.5, line_color="color_m", name='MSA')

tab1 = Panel(child=p12, title="12-month")
tab1.sizing_mode = 'scale_width'
tab2 = Panel(child=p3, title="3-month")
tab2.sizing_mode = 'scale_width'
tab3 = Panel(child=p1, title="1-month")
tab3.sizing_mode = 'scale_width'
tabs = Tabs(tabs=[tab1, tab2, tab3])

#p = gridplot([[tabs]], sizing_mode='scale_width', toolbar_location='below')

#p = layout([[tabs]])
tabs.sizing_mode = 'scale_width'

show(tabs)


script, div = components(tabs)
with open('C:/Working/bdecon.github.io/plots/unemp_map.html', 'w') as text_file:
    text_file.write(f'{script} {div}')

In [ ]:
def msa_calc(msa, data):
    '''Return labor market stats from BLS data when given BLS MSA code'''
    dfm = data[data['Code'] == msa]
    md = MSA[MSA['Code'] == msa]
    st = md['State'].iloc[-1]
    lt = dfm['Rate'].iloc[-1]
    ltd = dfm['date'].iloc[-1].strftime('%B %Y')
    m = dfm['Area'].iloc[-1]
    d12 = round(dfm.Rate.astype('float').diff(12).iloc[-1], 1)
    d3 = round(dfm.Rate.astype('float').rolling(3).mean().diff(3).iloc[-1], 1)
    d1 = round(dfm.Rate.astype('float').diff(1).iloc[-1], 1)
    c12 = ('limegreen' if d12 < -0.001 else 'red' if d12 > 0.001 else 'lightgray')
    c3 = ('limegreen' if d3 < -0.001 else 'red' if d3 > 0.001  else 'lightgray')
    c1 = ('limegreen' if d1 < -0.001 else 'red' if d1 > 0.001  else 'lightgray')
    s12 = (abs(d12) * 3.0 + 3 if abs(d12) > 0 else 3)
    s3 = (abs(d3) * 3.0 + 3 if abs(d3) > 0 else 3)
    s1 = (abs(d1) * 3.0 + 3 if abs(d1) > 0 else 3)
    lat = md['latitude'].iloc[-1]
    lon = md['longitude'].iloc[-1]
    if st == 'AK':
        lat = 0.35 * md['latitude'].iloc[-1] + 5
        lon = 0.35 * md['longitude'].iloc[-1] - 63
    if st == 'HI':
        lat = md['latitude'].iloc[-1] + 7
        lon = md['longitude'].iloc[-1] + 51
    sh12 = ('▼' if d12 < -0.001 else '▲' if d12 > 0.001 else '')
    sh3 = ('▼' if d3 < -0.001 else '▲' if d3 > 0.001 else '')
    sh1 = ('▼' if d1 < -0.001 else '▲' if d1 > 0.001 else '')
    return {'name': m, 'latest': lt, 'd12': d12, 'c12': c12, 's12': s12, 
            'd3': d3, 'c3': c3, 's3': s3, 'd1': d1, 'c1': c1, 's1': s1, 
            'lat': lat, 'lon': lon, 'date': ltd, 'sh12': sh12, 'sh3': sh3,
            'sh1': sh1}

In [ ]:
# CSV file contains city names, their BLS codes (Metropolitan 
# MTAs/NECTAs only), and their latitude and longitude
MSA = pd.read_csv('C:/Working/Python/Unemp_Map/MSA.csv')
MSA['Code'] = [m[1]['fullcode'][3:-2] for m in MSA.iterrows()]
MSA['State'] = [m[1]['fullname'][-2:] for m in MSA.iterrows()]
MSA.loc[:,:] = MSA[MSA['State'] != 'PR']

#url = 'https://www.bls.gov/web/metro/ssamatab1.txt'
url = 'ssamatab1.txt'

df = pd.read_table(url, sep='\s\s+', skiprows=3, skipfooter=4, engine='python')
data = df[df['Year'] >= 2017]
data['date'] = [pd.to_datetime(f'{v.Year}-{v.Month}-01') for i, v in data.iterrows()]

result = pd.DataFrame([msa_calc(msa, data) for msa in list(MSA.Code.unique())])

In [ ]:
# Get State Map Data
shpfile = 'C:/Working/Python/Unemp_Map/st99_d00.shp'
sf = shapefile.Reader(shpfile)
skip = ['Alaska', 'Hawaii', 'Puerto Rico']

rows=[]
for i, j in zip(sf.shapeRecords(), sf.shapes()):
    rows.append(i.record + [j.points])

others = {f'{r[5]}{r[3]}': {'lats': [i[1] for i in r[-1]], 
                            'lons': [i[0] for i in r[-1]], 
                            'name': r[5]} for r in rows if r[5] not in skip}

alaska = {f'{r[5]}{r[3]}': {'lats': [0.35 * i[1] + 5 for i in r[-1]], 
                            'lons': [0.35 * i[0] - 63 for i in r[-1]], 
                            'name': r[5]} for r in rows if r[5] == 'Alaska' 
                              and np.max([i[0] for i in r[-1]]) < 0}
    
hawaii = {f'{r[5]}{r[3]}': {'lats': [i[1] + 7 for i in r[-1]], 
                            'lons': [i[0] + 51 for i in r[-1]], 
                            'name': r[5]} for r in rows if r[5] == 'Hawaii' and r[0] > 0.005}

states = {**others, **hawaii, **alaska}

In [ ]:
# Small version

# Identify data sources
source = ColumnDataSource(data=dict(
    x = [state["lons"] for state in states.values()],
    y = [state["lats"] for state in states.values()],
    name = [state['name'] for state in states.values()]))

# Matching up MSA table with bokeh format
source2 = ColumnDataSource(data=dict(
        lat = list(result['lat'].values),
        lon = list(result['lon'].values),
        s12 = list(result['s12'].values),
        s3 = list(result['s3'].values),
        s1 = list(result['s1'].values),
        name = list(result['name'].values),
        color_ann = list(result['c12'].values),
        color_q = list(result['c3'].values),
        color_m = list(result['c1'].values),
        lt_val = list(result['latest'].values),
        d_ann = list(result['d12'].values),
        d_q = list(result['d3'].values),
        d_m = list(result['d1'].values),
        date = list(result['date'].values),
        sh_ann = list(result['sh12'].values),
        sh_q = list(result['sh3'].values),
        sh_m = list(result['sh1'].values)))

# HTML text that shows when hovering over an MSA
tooltips = """
    <div style="line-height: 110%;">
        <span style="font-size: 12px;">@name:</span><br>
        <span style="font-size: 11px; font-style: italic;">@date:</span>
        <div style="text-align: right;">
        <span style="font-size: 10px; padding: 0 5px;">Unemployed, looking for work: </span>
        <span style="display: inline-block; font-size: 11px; font-weight: bold; min-width: 26px;">@lt_val{1.1}%</span><br>
        <span style="font-size: 10px; padding: 0 5px;">One-year change (pp): </span>
        <span style="display: inline-block; color: @color_ann; width: 12px;">@sh_ann</span>
        <span style="display: inline-block; font-size: 11px; font-weight: bold; padding: 0 8px 0 0; width: 26px;">@d_ann{1.1}</span><br>
        <span style="font-size: 10px; padding: 0 5px;">Quarterly change (pp): </span>
        <span style="display: inline-block; color: @color_q; width: 12px;">@sh_q</span>
        <span style="display: inline-block; font-size: 11px; font-weight: bold; padding: 0 8px 0 0; width: 26px;">@d_q{1.1}</span><br>
        <span style="font-size: 10px; padding: 0 5px;">Monthly change (pp): </span>
        <span style="display: inline-block; color: @color_m; width: 12px;">@sh_m</span>
        <span style="display: inline-block; font-size: 11px; font-weight: bold; padding: 0 8px 0 0; width: 26px;">@d_m{1.1}</span></div>
    </div>
"""

hover = HoverTool(tooltips=tooltips, show_arrow=False, names=['MSA'])
TOOLS = 'pan, wheel_zoom, reset, save'

# Annual change
p12 = figure(tools=[TOOLS, hover], width=375, height=220,
    x_axis_location=None, y_axis_location=None)

p12.grid.grid_line_color = None
p12.outline_line_color = None
p12.patches('x', 'y', source=source, fill_color='white',
          fill_alpha=1.0, line_color='darkslategray', line_width=0.5)

p12.circle(x="lon", y="lat", size="s12", fill_color="color_ann", source=source2,
                fill_alpha=0.5, line_color="color_ann", name='MSA')

# Quarterly Change (3m avg over prev 3m avg)
p3 = figure(tools=[TOOLS, hover], width=375, height=220,
    x_axis_location=None, y_axis_location=None)

p3.grid.grid_line_color = None
p3.outline_line_color = None
p3.patches('x', 'y', source=source, fill_color='white',
          fill_alpha=1.0, line_color='darkslategray', line_width=0.5)

p3.circle(x="lon", y="lat", size="s3", fill_color="color_q", source=source2,
                fill_alpha=0.5, line_color="color_q", name='MSA')

# Monthly change
p1 = figure(tools=[TOOLS, hover], width=375, height=220, 
    x_axis_location=None, y_axis_location=None)

p1.grid.grid_line_color = None
p1.outline_line_color = None 
p1.patches('x', 'y', source=source, fill_color='white',
          fill_alpha=1.0, line_color='darkslategray', line_width=0.5)

p1.circle(x="lon", y="lat", size="s1", fill_color="color_m", source=source2,
                fill_alpha=0.5, line_color="color_m", name='MSA')

tab1 = Panel(child=p12, title="12-month")
tab2 = Panel(child=p3, title="3-month")
tab3 = Panel(child=p1, title="1-month")
tabs = Tabs(tabs=[tab1, tab2, tab3])

p = gridplot([[tabs]], sizing_mode='scale_width', toolbar_location='below')
show(p)

script, div = components(p)
with open('C:/Working/bdecon.github.io/plots/unemp_map_small.html', 'w') as text_file:
    text_file.write(f'{script} {div}')

In [ ]:
def msa_calc(msa, data):
    '''Return labor market stats from BLS data when given BLS MSA code'''
    dfm = data[data['Code'] == msa]
    md = MSA[MSA['Code'] == msa]
    st = md['State'].iloc[-1]
    lt = dfm['Rate'].iloc[-1]
    ltd = dfm['date'].iloc[-1].strftime('%B %Y')
    m = dfm['Area'].iloc[-1]
    d12 = round(dfm.Rate.astype('float').diff(12).iloc[-1], 1)
    d3 = round(dfm.Rate.astype('float').rolling(3).mean().diff(3).iloc[-1], 1)
    d1 = round(dfm.Rate.astype('float').diff(1).iloc[-1], 1)
    c12 = ('limegreen' if d12 < -0.001 else 'red' if d12 > 0.001 else 'lightgray')
    c3 = ('limegreen' if d3 < -0.001 else 'red' if d3 > 0.001  else 'lightgray')
    c1 = ('limegreen' if d1 < -0.001 else 'red' if d1 > 0.001  else 'lightgray')
    s12 = (abs(d12) * 5.0 + 4 if abs(d12) > 0 else 4)
    s3 = (abs(d3) * 5.0 + 4 if abs(d3) > 0 else 4)
    s1 = (abs(d1) * 5.0 + 4 if abs(d1) > 0 else 4)
    lat = md['latitude'].iloc[-1]
    lon = md['longitude'].iloc[-1]
    if st == 'AK':
        lat = 0.35 * md['latitude'].iloc[-1] + 5
        lon = 0.35 * md['longitude'].iloc[-1] - 63
    if st == 'HI':
        lat = md['latitude'].iloc[-1] + 7
        lon = md['longitude'].iloc[-1] + 51
    sh12 = ('▼' if d12 < -0.001 else '▲' if d12 > 0.001 else '')
    sh3 = ('▼' if d3 < -0.001 else '▲' if d3 > 0.001 else '')
    sh1 = ('▼' if d1 < -0.001 else '▲' if d1 > 0.001 else '')
    return {'name': m, 'latest': lt, 'd12': d12, 'c12': c12, 's12': s12, 
            'd3': d3, 'c3': c3, 's3': s3, 'd1': d1, 'c1': c1, 's1': s1, 
            'lat': lat, 'lon': lon, 'date': ltd, 'sh12': sh12, 'sh3': sh3,
            'sh1': sh1}

In [ ]:
# CSV file contains city names, their BLS codes (Metropolitan 
# MTAs/NECTAs only), and their latitude and longitude
MSA = pd.read_csv('C:/Working/Python/Unemp_Map/MSA.csv')
MSA['Code'] = [m[1]['fullcode'][3:-2] for m in MSA.iterrows()]
MSA['State'] = [m[1]['fullname'][-2:] for m in MSA.iterrows()]
MSA.loc[:,:] = MSA[MSA['State'] != 'PR']

#url = 'https://www.bls.gov/web/metro/ssamatab1.txt'
url = 'ssamatab1.txt'

df = pd.read_table(url, sep='\s\s+', skiprows=3, skipfooter=4, engine='python')
data = df[df['Year'] >= 2017]
data['date'] = [pd.to_datetime(f'{v.Year}-{v.Month}-01') for i, v in data.iterrows()]

result = pd.DataFrame([msa_calc(msa, data) for msa in list(MSA.Code.unique())])

In [ ]:
# Get State Map Data
shpfile = 'C:/Working/Python/Unemp_Map/st99_d00.shp'
sf = shapefile.Reader(shpfile)
skip = ['Alaska', 'Hawaii', 'Puerto Rico']

rows=[]
for i, j in zip(sf.shapeRecords(), sf.shapes()):
    rows.append(i.record + [j.points])

others = {f'{r[5]}{r[3]}': {'lats': [i[1] for i in r[-1]], 
                            'lons': [i[0] for i in r[-1]], 
                            'name': r[5]} for r in rows if r[5] not in skip}

alaska = {f'{r[5]}{r[3]}': {'lats': [0.35 * i[1] + 5 for i in r[-1]], 
                            'lons': [0.35 * i[0] - 63 for i in r[-1]], 
                            'name': r[5]} for r in rows if r[5] == 'Alaska' 
                              and np.max([i[0] for i in r[-1]]) < 0}
    
hawaii = {f'{r[5]}{r[3]}': {'lats': [i[1] + 7 for i in r[-1]], 
                            'lons': [i[0] + 51 for i in r[-1]], 
                            'name': r[5]} for r in rows if r[5] == 'Hawaii' and r[0] > 0.005}

states = {**others, **hawaii, **alaska}

In [ ]:
# Medium version

# Identify data sources
source = ColumnDataSource(data=dict(
    x = [state["lons"] for state in states.values()],
    y = [state["lats"] for state in states.values()],
    name = [state['name'] for state in states.values()]))

# Matching up MSA table with bokeh format
source2 = ColumnDataSource(data=dict(
        lat = list(result['lat'].values),
        lon = list(result['lon'].values),
        s12 = list(result['s12'].values),
        s3 = list(result['s3'].values),
        s1 = list(result['s1'].values),
        name = list(result['name'].values),
        color_ann = list(result['c12'].values),
        color_q = list(result['c3'].values),
        color_m = list(result['c1'].values),
        lt_val = list(result['latest'].values),
        d_ann = list(result['d12'].values),
        d_q = list(result['d3'].values),
        d_m = list(result['d1'].values),
        date = list(result['date'].values),
        sh_ann = list(result['sh12'].values),
        sh_q = list(result['sh3'].values),
        sh_m = list(result['sh1'].values)))

# HTML text that shows when hovering over an MSA
tooltips = """
    <div style="line-height: 110%;">
        <span style="font-size: 12px;">@name:</span><br>
        <span style="font-size: 11px; font-style: italic;">@date:</span>
        <div style="text-align: right;">
        <span style="font-size: 10px; padding: 0 5px;">Unemployed, looking for work: </span>
        <span style="display: inline-block; font-size: 11px; font-weight: bold; min-width: 26px;">@lt_val{1.1}%</span><br>
        <span style="font-size: 10px; padding: 0 5px;">One-year change (pp): </span>
        <span style="display: inline-block; color: @color_ann; width: 12px;">@sh_ann</span>
        <span style="display: inline-block; font-size: 11px; font-weight: bold; padding: 0 8px 0 0; width: 26px;">@d_ann{1.1}</span><br>
        <span style="font-size: 10px; padding: 0 5px;">Quarterly change (pp): </span>
        <span style="display: inline-block; color: @color_q; width: 12px;">@sh_q</span>
        <span style="display: inline-block; font-size: 11px; font-weight: bold; padding: 0 8px 0 0; width: 26px;">@d_q{1.1}</span><br>
        <span style="font-size: 10px; padding: 0 5px;">Monthly change (pp): </span>
        <span style="display: inline-block; color: @color_m; width: 12px;">@sh_m</span>
        <span style="display: inline-block; font-size: 11px; font-weight: bold; padding: 0 8px 0 0; width: 26px;">@d_m{1.1}</span></div>
    </div>
"""

hover = HoverTool(tooltips=tooltips, show_arrow=False, names=['MSA'])
TOOLS = 'pan, wheel_zoom, reset, save'

# Annual change
p12 = figure(tools=[TOOLS, hover], width=510, height=320,
    x_axis_location=None, y_axis_location=None)

p12.grid.grid_line_color = None
p12.outline_line_color = None
p12.patches('x', 'y', source=source, fill_color='white',
          fill_alpha=1.0, line_color='darkslategray', line_width=0.5)

p12.circle(x="lon", y="lat", size="s12", fill_color="color_ann", source=source2,
                fill_alpha=0.5, line_color="color_ann", name='MSA')

# Quarterly Change (3m avg over prev 3m avg)
p3 = figure(tools=[TOOLS, hover], width=510, height=320,
    x_axis_location=None, y_axis_location=None)

p3.grid.grid_line_color = None
p3.outline_line_color = None
p3.patches('x', 'y', source=source, fill_color='white',
          fill_alpha=1.0, line_color='darkslategray', line_width=0.5)

p3.circle(x="lon", y="lat", size="s3", fill_color="color_q", source=source2,
                fill_alpha=0.5, line_color="color_q", name='MSA')

# Monthly change
p1 = figure(tools=[TOOLS, hover], width=510, height=320, 
    x_axis_location=None, y_axis_location=None)

p1.grid.grid_line_color = None
p1.outline_line_color = None 
p1.patches('x', 'y', source=source, fill_color='white',
          fill_alpha=1.0, line_color='darkslategray', line_width=0.5)

p1.circle(x="lon", y="lat", size="s1", fill_color="color_m", source=source2,
                fill_alpha=0.5, line_color="color_m", name='MSA')

tab1 = Panel(child=p12, title="12-month")
tab2 = Panel(child=p3, title="3-month")
tab3 = Panel(child=p1, title="1-month")
tabs = Tabs(tabs=[tab1, tab2, tab3])

p = gridplot([[tabs]], sizing_mode='scale_width', toolbar_location='below')
show(p)

script, div = components(p)
with open('C:/Working/bdecon.github.io/plots/unemp_map_medium.html', 'w') as text_file:
    text_file.write(f'{script} {div}')